# ReAct Agent

Adapted from https://github.com/opendatahub-io/llama-stack-demos/blob/main/demos/rag_agentic/notebooks/Level3_advance_agentic_with_Prompt_Chaining_ReAct.ipynb

In [1]:
import rich
import uuid
from llama_stack_client import Agent, LlamaStackClient, AgentEventLogger
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

In [2]:
# this may be missing from pyproject.toml, add it if you need it
#!uv pip install geocoder

In [3]:
@client_tool
def get_location(query: str = "location"):
    """
    Provide the location upon request.

    :param query: The query from user
    :returns: Information about user location
    """
    import geocoder
    
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

In [4]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [5]:
models = client.models.list()
rich.print(models)

[
    Model(
        identifier='all-MiniLM-L6-v2',
        metadata={'embedding_dimension': 384.0},
        api_model_type='embedding',
        provider_id='ollama',
        provider_resource_id='all-minilm:latest',
        type='model',
        model_type='embedding'
    ),
    Model(
        identifier='meta-llama/Llama-3.2-3B-Instruct',
        metadata={},
        api_model_type='llm',
        provider_id='ollama',
        provider_resource_id='llama3.2:3b-instruct-fp16',
        type='model',
        model_type='llm'
    )
]

In [6]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

[
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        provider_resource_id='builtin::rag',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    ),
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        provider_resource_id='builtin::websearch',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    )
]

In [7]:
tools = client.tools.list()
rich.print(tools)

[
    Tool(
        description='Insert documents into memory',
        identifier='insert_into_memory',
        parameters=[],
        provider_id='rag-runtime',
        provider_resource_id='insert_into_memory',
        tool_host='distribution',
        toolgroup_id='builtin::rag',
        type='tool',
        metadata=None
    ),
    Tool(
        description='Search for information in a database.',
        identifier='knowledge_search',
        parameters=[
            Parameter(
                description='The query to search for. Can be a natural language sentence or keywords.',
                name='query',
                parameter_type='string',
                required=True,
                default=None
            )
        ],
        provider_id='rag-runtime',
        provider_resource_id='knowledge_search',
        tool_host='distribution',
        toolgroup_id='builtin::rag',
        type='tool',
        metadata=None
    ),
    Tool(
        description='Search the web for information',
        identifier='web_search',
        parameters=[
            Parameter(
                description='The query to search for',
                name='query',
                parameter_type='string',
                required=True,
                default=None
            )
        ],
        provider_id='tavily-search',
        provider_resource_id='web_search',
        tool_host='distribution',
        toolgroup_id='builtin::websearch',
        type='tool',
        metadata=None
    )
]

## Baseline

In [8]:
client.toolgroups.register(
    toolgroup_id="builtin::websearch",
    provider_id="tavily-search",
)

In [9]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

[
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        provider_resource_id='builtin::rag',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    ),
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        provider_resource_id='builtin::websearch',
        type='tool_group',
        args=None,
        mcp_endpoint=None
    )
]

In [10]:
agent = Agent(
    client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant.",
    tools=["builtin::websearch"],
)

In [11]:
response = agent.create_turn(
    messages=[{"role": "user", "content": "Are you able to search the web?"}],
    session_id=agent.create_session(f"tool-check-{uuid.uuid4()}"),
    stream=False,
)
rich.print(response)

Turn(
    input_messages=[UserMessage(content='Are you able to search the web?', role='user', context=None)],
    output_message=CompletionMessage(
        content='Yes, I can search the web for information on a wide range of topics. What would you like me to 
look up?',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    session_id='ccbed6cc-eec9-4fdb-acde-d87763dae77e',
    started_at=datetime.datetime(2025, 5, 12, 13, 25, 45, 680943, tzinfo=TzInfo(UTC)),
    steps=[
        InferenceStep(
            api_model_response=CompletionMessage(
                content='Yes, I can search the web for information on a wide range of topics. What would you like 
me to look up?',
                role='assistant',
                stop_reason='end_of_turn',
                tool_calls=[]
            ),
            step_id='4fb0c171-3895-418f-9cd9-ca4d06869805',
            step_type='inference',
            turn_id='8699252e-bb45-4a41-a1f8-6dd12b24a092',
            completed_at=datetime.datetime(2025, 5, 12, 13, 25, 46, 945354, tzinfo=TzInfo(UTC)),
            started_at=datetime.datetime(2025, 5, 12, 13, 25, 45, 681050, tzinfo=TzInfo(UTC))
        )
    ],
    turn_id='8699252e-bb45-4a41-a1f8-6dd12b24a092',
    completed_at=datetime.datetime(2025, 5, 12, 13, 25, 46, 956020, tzinfo=TzInfo(UTC)),
    output_attachments=[]
)

In [12]:
agent = Agent(
    client, 
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant. Use websearch tool to help answer questions.",
    tools=["builtin::websearch"],
)
user_prompts = [
    # "Are there any weather-related risks in my area?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",

]
session_id = agent.create_session(f"test-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")

    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )
    for log in AgentEventLogger().log(response):
        log.print()

User> Are there any weather-related risks in my area that could disrupt network connectivity or system 
availability?

inference> brave_search.call(query="weather-related risks near me")
tool_execution> Tool:brave_search Args:{'query': 'weather-related risks near me'}
tool_execution> Tool:brave_search Response:{"query": "weather-related risks near me", "top_k": [{"title": "Weather in current location", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Current', 'region': 'Harbour Island', 'country': 'Bahamas', 'lat': 25.4333, 'lon': -76.7833, 'tz_id': 'America/Nassau', 'localtime_epoch': 1747056156, 'localtime': '2025-05-12 09:22'}, 'current': {'last_updated_epoch': 1747055700, 'last_updated': '2025-05-12 09:15', 'temp_c': 24.6, 'temp_f': 76.3, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 19.2, 'wind_kph': 31.0, 'wind_degree': 117, 'wind_dir': 'ESE', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 27.6, 'feelslike_f':

## Prompt Chaining

In [13]:
agent = Agent(
    client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="""You are a helpful assistant. 
    When a user asks about their location, use the get_location tool. 
    When searching for nearby places, use the websearch tool.
    """,
    tools=[get_location, "builtin::websearch"],
)
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"prompt-chaining-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

User> Where am I?

inference> [get_location(query="my current location")]
tool_execution> Tool:get_location Args:{'query': 'my current location'}
tool_execution> Tool:get_location Response:"Your current location is: London, England, GB"
inference> {"location": "London, England, GB"}<|python_tag|>{"location": "London, England, GB"}


User> Are there any weather-related risks in my area?

inference> [get_location(query="weather risks near me")]
tool_execution> Tool:get_location Args:{'query': 'weather risks near me'}
tool_execution> Tool:get_location Response:"Your current location is: London, England, GB"
inference> "There are no severe weather warnings for your location at this time. However, it's always a good idea to check the forecast regularly for any updates."


### ReAct Agent 

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

For example, when asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent:
1. **Reasons** it needs location information first
2. **Acts** by calling the `get_location` client tool
3. **Observes** the location result
4. **Reasons** about the next step
5. **Acts** by calling the `websearch` tool with observed location
6. **Observes** and processes the search results

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [ ]:
agent = ReActAgent(
    client=client,
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant. Use the tools at your disposal to answer any questions.",
    tools=[get_location, "builtin::websearch"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
)
user_prompts = [
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"web-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

User> Are there any weather-related risks in my area?

inference> 

{"thought": "Are there any weather-related risks in my area?", "action": null, "answer": "No specific information available."}
